In [ ]:
7.1. Meet the DS Lab Applicants

When you decided to start down the path to becoming a data scientist at WQU, the first thing you did was to register an account with us. Then you took our admissions exam test, and began your data science journey! But not everyone who creates an account takes the admissions exam. Is there a way to improve that completion rate?

In this project, you'll help run an experiment to see if sending a reminder email to applicants can increase the likelihood that they'll complete the admissions exam. This type of experiment is called a hypothesis test or an A/B test.

In this lesson, we'll try to get a better sense of what kind of people sign up for Applied Data Science Lab — where they're from, how old are they, what have they previously studied, and more.

Data Ethics: This project is based on a real experiment that the WQU data science team conducted in June of 2022 There is, however, one important difference. While the data science team used real student data, you're going to use synthetic data. It is designed to have characteristics that are similar to the real thing without exposing any actual personal data — like names, birthdays, and email addresses — that would violate our students' privacy.
from pprint import PrettyPrinter
​
import pandas as pd
import plotly.express as px
import wqet_grader
from country_converter import CountryConverter
from IPython.display import VimeoVideo
from pymongo import MongoClient
​
wqet_grader.init("Project 7 Assessment")
​
VimeoVideo("733383823", h="d6228d4de1", width=600)
The DS Lab student data is stored in a MongoDB database. So we'll start the lesson by creating a PrettyPrinter, and connecting to the right database and collection.

VimeoVideo("733383369", h="4d221e7fb7", width=600)
Task 7.1.1: Instantiate a PrettyPrinter, and assign it to the variable pp.

Construct a PrettyPrinter instance in pprint.
pp = PrettyPrinter(indent=2)
print("pp type:", type(pp))
pp type: <class 'pprint.PrettyPrinter'>
Next up, let's connect to the MongoDB server.

Connect
VimeoVideo("733383007", h="13b2c716ac", width=600)
Task 7.1.2: Create a client that connects to the database running at localhost on port 27017.

What's a database client?
What's a database server?
Create a client object for a MongoDB instance.
client = MongoClient(host="Localhost", port=27017)
print("client type:", type(client))
client type: <class 'pymongo.mongo_client.MongoClient'>
Okay! Let's take a look at the databases that are available to us.

Task 7.1.3: Print a list of the databases available on client.

What's an iterator?
List the databases of a server using PyMongo.
Print output using pprint.
pp.pprint(list(client.list_databases()))
[ {'empty': False, 'name': 'admin', 'sizeOnDisk': 40960},
  {'empty': False, 'name': 'air-quality', 'sizeOnDisk': 4190208},
  {'empty': False, 'name': 'config', 'sizeOnDisk': 12288},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 73728},
  {'empty': False, 'name': 'wqu-abtest', 'sizeOnDisk': 585728}]
We're interested in the "wqu-abtest" database, so let's assign a variable and get moving.

By the way, did you notice our old friend the air quality data? Isn't it nice to know that if you ever wanted to go back and do those projects again, the data will be there waiting for you?

VimeoVideo("733382605", h="e0b87a5ff8", width=600)
Task 7.1.4: Assign the "ds-applicants" collection in the "wqu-abtest" database to the variable name ds_app.

What's a MongoDB collection?
Access a collection in a database using PyMongo.
db = client["wqu-abtest"]
ds_app = db["ds-applicants"]
print("ds_app type:", type(ds_app))
ds_app type: <class 'pymongo.collection.Collection'>
Now let's take a look at what we've got. First, let's find out how many applicants are currently in our collection.

Explore
VimeoVideo("733382346", h="9da7d3d1d8", width=600)
Task 7.1.5: Use the count_documents method to see how many documents are in the ds_app collection.

What's a MongoDB document?
Count the documents in a collection using PyMongo.
Warning: The exact number of documents in the database has changed since this video was filmed. So don't worry if you don't get exactly the same numbers as the instructor for the tasks in this project.
# Count documents in `ds_app`
n_documents = ds_app.count_documents({})
print("Num. documents in 'ds-applicants':", n_documents)
Num. documents in 'ds-applicants': 5025
So that's the number of individual records in the collection, but what do those records look like? The last time we did anything with a MongoDB database, the data was semi-structured, and that's true here as well. Recall that semi-structured data is arranged according to some kind of logic, but it can't be displayed in a regular table of rows and columns.

Let's take a look at how these documents are laid out.

VimeoVideo("733380658", h="a7988083f4", width=600)
Task 7.1.6: Use the find_one method to retrieve one document from the ds_app collection and assign it to the variable name result.

What's semi-structured data?
Retrieve a document from a collection using PyMongo.
result = ds_app.find_one({})
print("result type:", type(result))
pp.pprint(result)
result type: <class 'dict'>
{ '_id': ObjectId('6525d787953844722c8383f8'),
  'admissionsQuiz': 'incomplete',
  'birthday': datetime.datetime(1998, 4, 29, 0, 0),
  'countryISO2': 'GB',
  'createdAt': datetime.datetime(2022, 5, 13, 15, 2, 44),
  'email': 'terry.hassler28@yahow.com',
  'firstName': 'Terry',
  'gender': 'male',
  'highestDegreeEarned': "Bachelor's degree",
  'lastName': 'Hassler'}
See why we shouldn't be using the real data for an assignment like this? Each document includes the applicant's birthday, country of origin, email address, first and last name, and their highest level of educational attainment — all things that would make our students readily identifiable. Good thing we've got synthetic data instead!WQU WorldQuant University Applied Data Science Lab QQQQ

Nationality
Let's start the analysis. One of the possibilities in each record is the country of origin. We already know WQU is a pretty diverse place, but we can figure out just how diverse it is by seeing where applicants are coming from.

First, we'll perform an aggregation to count countries.

VimeoVideo("733379562", h="8ffd2458e0", width=600)
Task 7.1.7: Use the aggregate method to calculate how many applicants there are from each country.

Perform aggregation calculations on documents using PyMongo.
Tip: ISO stands for "International Organization for Standardization". So, when you write your query, make sure you're not confusing the letter O with the number 0.

result = ds_app.aggregate(
    [
        {
            "$group": {"_id": "$countryISO2", "count": {"$count": {}}
            
            }
        }
    
    ]
​
)
print("result type:", type(result))
result type: <class 'pymongo.command_cursor.CommandCursor'>
Next, we'll create and print a DataFrame with the results.

VimeoVideo("733376898", h="fc7f30e75a", width=600)
Task 7.1.8: Put your results from the previous task into a DataFrame named df_nationality. Your DataFrame should have two columns: "country_iso2" and "count". It should be sorted from the smallest to the largest value of "count".

Create a DataFrame from a dictionary using pandas.
Rename a Series in pandas.
Sort a DataFrame or Series in pandas.
df_nationality = (
    pd.DataFrame(result).rename({"_id": "country_iso2"},axis="columns").sort_values("count")
)
print("df_nationality type:", type(df_nationality))
print("df_nationality shape", df_nationality.shape)
df_nationality.head()
df_nationality type: <class 'pandas.core.frame.DataFrame'>
df_nationality shape (139, 2)
country_iso2	count
0	BB	1
121	TM	1
129	GY	1
32	RO	1
85	AD	1
Tip: If you see that there's no data in df_nationality, it's likely that there's an issue with your query in the previous task.

Now we have the countries, but they're represented using the ISO 3166-1 alpha-2 standard, where each country has a two-letter code. It'll be much easier to interpret our data if we have the full country name, so we'll need to do some data enrichment using country converter library.

Since country_converter is an open-source library, there are several things to think about before we can bring it into our project. The first thing we need to do is figure out if we're even allowed to use the library for the kind of project we're working on by taking a look at the library's license. country_converter has a GNU General Public License, so there are no worries there.

Second, we need to make sure the software is being actively maintained. If the last time anybody changed the library was back in 2014, we're probably going to run into some problems when we try to use it. country_converter's last update is very recent, so we aren't going to have any trouble there either.

Third, we need to see what kinds of quality-control measures are in place. Even if the library was updated five minutes ago and includes a license that gives us permission to do whatever we want, it's going to be entirely useless if it's full of mistakes. Happily, country_converter's testing coverage and build badges look excellent, so we're good to go there as well.

The last thing we need to do is make sure the library will do the things we need it to do by looking at its documentation. country_converter's documentation is very thorough, so if we run into any problems, we'll almost certainly be able to figure out what went wrong.

country_converter looks good across all those dimensions, so let's put it to work!

VimeoVideo("733373453", h="f8e954db9f", width=600)
Task 7.1.9: Instantiate a CountryConverter object named cc, and then use it to add a "country_name" column to the DataFrame df_nationality.

Convert country names from one format to another using country converter.
Create new columns derived from existing columns in a DataFrame using pandas.
cc = CountryConverter()
df_nationality["country_name"] = cc.convert(
    df_nationality["country_iso2"], to="name_short"
)
​
print("df_nationality shape:", df_nationality.shape)
df_nationality.head()
df_nationality shape: (139, 3)
country_iso2	count	country_name
0	BB	1	Barbados
121	TM	1	Turkmenistan
129	GY	1	Guyana
32	RO	1	Romania
85	AD	1	Andorra
That's better. Okay, let's turn that data into a bar chart.

VimeoVideo("733372561", h="2659ff0dc7", width=600)
Task 7.1.10: Create a horizontal bar chart of the 10 countries with the largest representation in df_nationality. Be sure to label your x-axis "Frequency [count]", your y-axis "Country", and use the title "DS Applicants by Country".

What's a bar chart?
Create a bar chart using plotly express.
# Create horizontal bar chart
fig = px.bar(
    data_frame=df_nationality.tail(10),
    x="count",
    y="country_name",
    orientation="h",
    title="DS Applicants: Nationality"
)
    # Set axis labels
fig.update_layout(xaxis_title="Frequency [count]", yaxis_title="Country")
fig.show()

That's showing us the raw number of applicants from each country, but since we're working with admissions data, it might be more helpful to see the proportion of applicants each country represents. We can get there by normalizing the dataset.

VimeoVideo("733371952", h="a061e33ab8", width=600)
Task 7.1.11: Create a "count_pct" column for df_nationality that shows the proportion of applicants from each country.

Create new columns derived from existing columns in a DataFrame using pandas.
df_nationality["count_pct"] = (df_nationality["count"]/df_nationality["count"].sum()) * 100
​
print("df_nationality shape:", df_nationality.shape)
df_nationality.head()
df_nationality shape: (139, 4)
country_iso2	count	country_name	count_pct
0	BB	1	Barbados	0.0199
121	TM	1	Turkmenistan	0.0199
129	GY	1	Guyana	0.0199
32	RO	1	Romania	0.0199
85	AD	1	Andorra	0.0199
Now we can turn that into a new bar chart.

VimeoVideo("733371556", h="7cae7252a8", width=600)
Task 7.1.12: Recreate your horizontal bar chart of the 10 countries with the largest representation in df_nationality, this time with the percentages. Be sure to label your x-axis "Frequency [%]", your y-axis "Country", and use the title "DS Applicants by Country".

What's a bar chart?
Create a bar chart using plotly express.
# Create horizontal bar chart
fig = px.bar(
    data_frame=df_nationality.tail(10),
    x="count_pct",
    y="country_name",
    orientation="h",
    title="DS Applicants: Nationality"
)
    # Set axis labels
fig.update_layout(xaxis_title="Frequency [%]", yaxis_title="Country")
fig.show()

Bar charts are useful, but since we're talking about actual places here, let's see how this data looks when we put it on a world map. However, plotly express requires the ISO 3166-1 alpha-3 codes. This means that we'll need to add another column to our DataFrame before we can make our visualization.

VimeoVideo("733370726", h="2b21ee76d2", width=600)
Task 7.1.13: Add a column named "country_iso3" to df_nationality. It should contain the 3-letter ISO abbreviation for each country in "country_iso2".

Create new columns derived from existing columns in a DataFrame using pandas.
df_nationality["country_iso3"] = cc.convert(
    df_nationality["country_iso2"], to="ISO3"
)
​
print("df_nationality shape:", df_nationality.shape)
df_nationality.head()
df_nationality shape: (139, 5)
country_iso2	count	country_name	count_pct	country_iso3
0	BB	1	Barbados	0.0199	BRB
121	TM	1	Turkmenistan	0.0199	TKM
129	GY	1	Guyana	0.0199	GUY
32	RO	1	Romania	0.0199	ROU
85	AD	1	Andorra	0.0199	AND
Perfect! Let's turn the table into a map!

VimeoVideo("733369606", h="73a380a6c6", width=600)
Task 7.1.14: Create a function build_nat_choropleth that returns plotly choropleth map showing the "count" of DS applicants in each country in the globe. Be sure to set your projection to "natural earth", and color_continuous_scale to px.colors.sequential.Oranges.

What's a choropleth map?
Create a choropleth map using plotly express.
def build_nat_choropleth():
    fig = px.choropleth(
    data_frame=df_nationality,
    locations="country_iso3",
    color="count_pct",
    projection="natural earth",
    color_continuous_scale=px.colors.sequential.Oranges,
    title="DS Applicants: Nationality"
)
    return fig
​
​
nat_fig = build_nat_choropleth()
print("nat_fig type:", type(nat_fig))
nat_fig.show()
nat_fig type: <class 'plotly.graph_objs._figure.Figure'>

Note: Political borders are subject to change, debate and dispute. As such, you may see borders on this map that you don't agree with. The political boundaries you see in Plotly are based on the Natural Earth dataset. You can learn more about their disputed boundaries policy here.

Cool! This is showing us what we knew already: most of the applicants come from Nigeria, India, and Pakistan. But this visualization also shows the global diversity of DS Lab students. Almost every country is represented in our student body!

Age
Now that we know where the applicants are from, let's see what else we can learn. For instance, how old are DS Lab applicants? We know the birthday of all our applicants, but we'll need to perform another aggregation to calculate their ages. We'll use the "$birthday" field and the "$$NOW" variable.

VimeoVideo("733367865", h="6e444cb810", width=600)
Task 7.1.15: Use the aggregate method to calculate the age for each of the applicants in ds_app. Store the results in result.

Perform aggregation calculations on documents using PyMongo.
Aggregate data using the $project operator in PyMongo.
Calculate the difference between dates using the $dateDiff operator in PyMongo.
#with aggregate you have () [] {}
result = ds_app.aggregate(
    [
        {
            "$project": {
                "years": {
                    "$dateDiff": {
                        "startDate": "$birthday",
                        "endDate": "$$NOW",
                        "unit": "year"
                        
                    }
                }
            }
        
        }
    
    ]
)
​
print("result type:", type(result))
result type: <class 'pymongo.command_cursor.CommandCursor'>
Once we have the query results, we can put them into a Series.

VimeoVideo("733367340", h="2b926b1e3a", width=600)
Task 7.1.16: Read your result from the previous task into a DataFrame, and create a Series called ages.

Create a Series in pandas.
ages = pd.DataFrame(result)["years"]
​
print("ages type:", type(ages))
print("ages shape:", ages.shape)
ages.head()
ages type: <class 'pandas.core.series.Series'>
ages shape: (5025,)
0    27
1    26
2    31
3    41
4    35
Name: years, dtype: int64
And finally, plot a histogram to show the distribution of ages.

VimeoVideo("733366740", h="bb14c884bb", width=600)
Task 7.1.17: Create function build_age_hist that returns a plotly histogram of ages. Be sure to label your x-axis "Age", your y-axis "Frequency [count]", and use the title "Distribution of DS Applicant Ages".

What's a histogram?
Create a histogram using plotly express
def build_age_hist():
    # Create histogram of `ages`
    fig = px.histogram(x=ages, nbins=20, title="Distribution of DS Applicant Ages")
    # Set axis labels
    fig.update_layout(xaxis_title="Age", yaxis_title="Frequency [count]")
    return fig
​
​
age_fig = build_age_hist()
print("age_fig type:", type(age_fig))
age_fig.show()
age_fig type: <class 'plotly.graph_objs._figure.Figure'>

It looks like most of our applicants are in their twenties, but we also have applicants in their 70s. What a wonderful example of lifelong learning. Role models for all of us!

Education
Okay, there's one more attribute left for us to explore: educational attainment. Which degrees do our applicants have? First, let's count the number of applicants in each category...

VimeoVideo("733366435", h="c6d3a83830", width=600)
Task 7.1.18: Use the aggregate method to calculate value counts for highest degree earned in ds_app.

Aggregate data in a series using value_counts in pandas.
result = ds_app.aggregate(
    [
        {
            "$group": {
                "_id": "$highestDegreeEarned",
                "count": {"$count": {}}
            }
        }
    ]
)
​
print("result type:", type(result))
result type: <class 'pymongo.command_cursor.CommandCursor'>
... and create a Series...

VimeoVideo("733365459", h="5c14d30a9e", width=600)
Task 7.1.19: Read your result from the previous task into a Series education.

Create a Series in pandas.
education = (
    pd.DataFrame(result)
    .rename({"_id": "highest_degree_earned"}, axis="columns")
    .set_index("highest_degree_earned")
    .squeeze()
)
​
print("education type:", type(education))
print("education shape:", education.shape)
education.head()
education type: <class 'pandas.core.series.Series'>
education shape: (5,)
highest_degree_earned
High School or Baccalaureate     832
Master's degree                  862
Doctorate (e.g. PhD)              76
Bachelor's degree               2643
Some College (1-3 years)         612
Name: count, dtype: int64
... and... wait! We need to sort these categories more logically. Since we're talking about the highest level of education our applicants have, we need to sort the categories hierarchically rather than alphabetically or numerically. The order should be: "High School or Baccalaureate", "Some College (1-3 years)", "Bachelor's Degree", "Master's Degree", and "Doctorate (e.g. PhD)". Let's do that with a function.

VimeoVideo("733362518", h="90dd9a3394", width=600)
#{k: v for v, k in enumerate(degrees)} #k for keys and v for values in dictionary
​
[2, 1]
Task 7.1.20: Complete the ed_sort function below so that it can be used to sort the index of education. When you're satisfied that you're going to end up with a properly-sorted Series, submit your code to the grader.

What's a dictionary comprehension?
Sort a DataFrame or Series in pandas.
def ed_sort(counts):
    """Sort array `counts` from highest to lowest degree earned."""
    degrees = [
        "High School or Baccalaureate",
        "Some College (1-3 years)",
        "Bachelor's degree",
        "Master's degree",
        "Doctorate (e.g. PhD)",
    ]
    mapping = {k: v for v, k in enumerate(degrees)}
    sort_order = [mapping[c] for c in counts]
    return sort_order
​
​
​
#ed_sort(["Bachelor's degree", "Some College (1-3 years)"])
education.sort_index(key=ed_sort, inplace=True)
education
highest_degree_earned
High School or Baccalaureate     832
Some College (1-3 years)         612
Bachelor's degree               2643
Master's degree                  862
Doctorate (e.g. PhD)              76
Name: count, dtype: int64
wqet_grader.grade("Project 7 Assessment", "Task 7.1.20", education)
​
Yup. You got it.

Score: 1

Now we can make a bar chart showing the educational attainment of the applicants. Make sure the levels are sorted correctly!

VimeoVideo("733360047", h="b17fffc11b", width=600)
#education has the values and df_education has the percentage 
education.head()
highest_degree_earned
High School or Baccalaureate     832
Some College (1-3 years)         612
Bachelor's degree               2643
Master's degree                  862
Doctorate (e.g. PhD)              76
Name: count, dtype: int64
Task 7.1.21: Create a function build_ed_bar that returns a plotly horizontal bar chart of education. Be sure to label your x-axis "Frequency [count]", y-axis "Highest Degree Earned", and use the title "DS Applicant Education Levels".

What's a bar chart?
Create a bar chart using plotly express.
#transfroming the series data to datafram and using a to make count a percentage
df_education = pd.DataFrame(education)
df_education = (df_education["count"]/df_education["count"].sum()) * 100
df_education
#to use value change x in the bar chart to education and for % change x to df_education
highest_degree_earned
High School or Baccalaureate    16.557214
Master's degree                 17.154229
Doctorate (e.g. PhD)             1.512438
Bachelor's degree               52.597015
Some College (1-3 years)        12.179104
Name: count, dtype: float64
def build_ed_bar():
    # Create bar chart
    fig = px.bar(
        x=df_education,
        y=education.index,
        orientation="h",
        title="DS Applications: Highest Degree Earned"
    )  
    # Add axis labels
    fig.update_layout(xaxis_title="Frequency [count]", yaxis_title="Higest Degree Earned")
    return fig
​
​
ed_fig = build_ed_bar()
print("ed_fig type:", type(ed_fig))
ed_fig.show()
ed_fig type: <class 'plotly.graph_objs._figure.Figure'>

